In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
%cd /drive/MyDrive/'Speaker-Diarization-ATML2021'/lifelong
%ls

/drive/MyDrive/Speaker-Diarization-ATML2021/lifelong
all_transcripts.json          icsimeetingcorpus@     __pycache__/
ge2eloss.py                   lifelong.ipynb         share@
helper_funcs.py               model.py               show_generator.py
hybrid_speaker_clustering.py  nearest_embeddings.py  spectral_cluster.py


In [ ]:
!pip install simpleder

In [ ]:
import tensorflow as tf
import tensorflow.keras as K
import simpleder
import time
import copy
import json
import os

from show_generator import *
from helper_funcs import *
from model import *
from ge2eloss import *
from hybrid_speaker_clustering import *
from nearest_embeddings import *

In [ ]:
n_mfcc = 13
show_len = 10 #minutes
icsi_audio_dir = "icsimeetingcorpus/Signals"
icsi_segments_dir = "icsimeetingcorpus/ICSIplus/Segments"
cache_dir = "/content/sample_data"

In [ ]:

if os.path.isfile('all_transcripts.json'):
    with open('all_transcripts.json', 'r') as json_file:
        all_transcripts = json.load(json_file)
else:
    all_transcripts = gather_transcripts(icsi_segments_dir)
    with open('all_transcripts.json', 'w') as json_file:
        json.dump(all_transcripts, json_file)

cleaned_transcripts = {k: remove_overlaps(all_transcripts[k], min_len=0) for k in all_transcripts.keys()}
small_transcripts = split_transcripts(cleaned_transcripts, show_len)


In [ ]:
# feature extractor
import librosa
def extract_mfcc(signal_segment, sr=16000, n_mfcc=13, n_fft=512):
    hop = int(0.010 * sr) # 10ms
    win = int(0.025 * sr) # 25ms
    mfcc = librosa.feature.mfcc(signal_segment, sr, n_mfcc= n_mfcc, hop_length= hop, win_length= win, n_fft=n_fft, window= "ham")
    del_mfcc = librosa.feature.delta(mfcc)
    feat = np.vstack([mfcc, del_mfcc])
    return feat

In [ ]:
# load the model
lifelong_model = K.models.load_model("share/embd_model.h5")
original_w = copy.deepcopy(lifelong_model.trainable_variables)

# generator takes only one walk through the dataset
generator = show_generator(small_transcripts, extract_mfcc, icsi_audio_dir, cache_dir, sr=16000, prefetch_buffer_size=25, max_threads=15)
int_to_token = generator.get_int_to_token()

centroids = []
n_embds = []
covariance_matrices = []
old_mfccs = []
old_labels = []


# lifelong Training 
model_opt = K.optimizers.RMSprop(learning_rate=1e-4)
param_opt = K.optimizers.RMSprop(learning_rate=1e-4)

w = tf.Variable(initial_value= 13.0, trainable= True, 
                constraint= lambda x: tf.clip_by_value(x, 0.0, 1000.0), name= 'w')    
b = tf.Variable(initial_value= -5.0, trainable= True, name= 'b')

der_record = []
max_mem = 500
for show in generator.dict_generator():
    mfcc_features = show['feats']
    rounded_groundtruth = show['labels']

    # # remove the overlap/non-speech segments
    # mask = tf.equal(rounded_groundtruth, 58)
    # mfcc_features = tf.boolean_mask(mfcc_features, mask)

    # concatenate mfcc features with older ones in memory
    original_length = len(mfcc_features)
    if len(old_mfccs) > 1: mfcc_features = tf.concat([mfcc_features, old_mfccs], axis=0)

    # put mfcc features through model
    with tf.GradientTape() as tape:

        tape.watch(w)
        tape.watch(b)
        print(mfcc_features.shape, len(old_mfccs))
        embeds = lifelong_model(mfcc_features, training=True)
        embeds = tf.nn.l2_normalize(embeds, axis=1)

        # get labels from clustering stuffs
        predicted_labels = []
        for i in range(len(embeds) - len(old_mfccs)):
            label, centroids, covariance_matrices, n_embds = classify_new_embeds(
                tf.expand_dims(embeds[i],0).numpy(), centroids, covariance_matrices, n_embds,
                min_threshold = 1.2, max_threshold = 1.5)
            predicted_labels.append(label)

        if len(old_labels)> 1: _labels = tf.concat([predicted_labels, old_labels], axis=0)
        else: _labels = predicted_labels

        # compute loss with ge2e
        embeds= tf.expand_dims(embeds, 0)
        _labels = tf.expand_dims(_labels, 0)
        loss0 = ge2e(embeds, _labels, [tf.squeeze(centroids)], w, b)

        # assuming that the model weights staw around the older weights
        temp = []
        for cw, ow in zip(lifelong_model.trainable_variables, original_w):
            temp.append(tf.reduce_sum(tf.square(cw-ow)))
        loss1 = tf.reduce_sum(temp)
        loss = loss0 + 1000*loss1

    # apply gradients
    gradients = tape.gradient(loss, [lifelong_model.trainable_variables, w, b])
    gradients1 = [(tf.clip_by_value(grad, -10.0, 10.0)) for grad in gradients[0]]
    gradients2 = [(tf.clip_by_value(grad, -10.0, 10.0)) for grad in gradients[1:]]
    model_opt.apply_gradients(zip(gradients1, lifelong_model.trainable_variables))
    param_opt.apply_gradients(zip(gradients2, [w,b]))

    # update old stuffs
    unique_classes = np.unique(predicted_labels)
    embeds = tf.squeeze(embeds)[0:original_length]
    for k in unique_classes:
        class_mask = tf.equal(predicted_labels, k)
        class_embeds = tf.boolean_mask(embeds, class_mask)
        class_mfccs = tf.boolean_mask(mfcc_features[0:original_length], class_mask)
        indices = identify_nearest_embeddings(class_embeds, 3)
        for i in indices:
            old_mfccs.append(class_mfccs[int(i)])
            old_labels.append(k)
            if len(old_mfccs) > max_mem: 
                old_mfccs.pop(0)
                old_labels.pop(0)

    # we may use the approximate ground truth to get DER
    hyp = []
    for i, label in enumerate(predicted_labels):
        if rounded_groundtruth[i] == 58: pass # dont evaluate on overlap
        hyp.append((str(label), float(i)*2, float(i+1)*2))

    ref = []
    for i, gt in enumerate(rounded_groundtruth):
        if rounded_groundtruth[i] == 58: pass # dont evaluate on overlap
        ref.append((str(gt), float(i)*2, float(i+1)*2))

    der = simpleder.DER(ref, hyp)
    der_record.append(der)

    print("loss: ", loss0.numpy(), "loss1: ", loss1.numpy(), "der: ", der) 

(300, 201, 26) 0
loss:  2.6698005 loss1:  0.0 der:  0.38717948717948714
(306, 201, 26) 6
loss:  0.7874465 loss1:  0.10133819 der:  0.4230769230769231
(312, 201, 26) 12
loss:  0.7209669 loss1:  1.773624e-05 der:  0.41025641025641024
(318, 201, 26) 18
loss:  1.4011935 loss1:  4.0464824e-05 der:  0.4512820512820513
(324, 201, 26) 24
loss:  1.2784088 loss1:  0.00010997161 der:  0.46923076923076923
(330, 201, 26) 30
loss:  176.25977 loss1:  0.0001592817 der:  0.38974358974358975
(336, 201, 26) 36
loss:  1.691704 loss1:  0.012703052 der:  0.43589743589743585
(342, 201, 26) 42
loss:  1.1151887 loss1:  0.0020570662 der:  0.34102564102564104
(348, 201, 26) 48
loss:  0.86249125 loss1:  0.000727457 der:  0.4923076923076923
(354, 201, 26) 54
loss:  180.58188 loss1:  0.0003042342 der:  0.4230769230769231
(360, 201, 26) 60
loss:  142.72243 loss1:  0.01052021 der:  0.34102564102564104
(366, 201, 26) 66
loss:  148.28365 loss1:  0.01589651 der:  0.34615384615384615
(372, 201, 26) 72
loss:  128.19917 lo

Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/drive/MyDrive/Speaker-Diarization-ATML2021/lifelong/show_generator.py", line 184, in run
    batch = self._sequencial_make_batch()
  File "/drive/MyDrive/Speaker-Diarization-ATML2021/lifelong/show_generator.py", line 164, in _sequencial_make_batch
    feats = self.interval_loader(intervals, fname)
  File "/drive/MyDrive/Speaker-Diarization-ATML2021/lifelong/show_generator.py", line 141, in <lambda>
    intervals, fname, audio_dir, featextr_fn, cache_dir, sr
  File "/drive/MyDrive/Speaker-Diarization-ATML2021/lifelong/show_generator.py", line 113, in interval_loader
    assert len(x) > int(np.round(start * sr)), f"{start} {stop} {len(x)/sr} {fname}"
AssertionError: 2124 2126 2122.552 Bro025



KeyboardInterrupt: ignored

In [14]:
np.mean(der_record)

0.42465837555657915